<a href="https://colab.research.google.com/github/HenryZumaeta/ReprojectCoordinates/blob/main/ReprojectCoordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='729D2D'>Reproject WGS84 Coordinates from UTM to Geographic- Shared by [Henry Zumaeta](https://github.com/HenryZumaeta)</font>**





<center>
  <img src='https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/UTM_to_Geo_1.png' height="120" alt="UTM2Geo-logo"/>
</center>


---


<center>
  <img src='https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/Coordenadas1.png' height="400" alt="Coordenadas-img"/>
</center>

**<font color='729D2D'><h2>Descripción:</h2></font>**

<font color='3A95B5'><h4>El archivo CSV debe tener las columnas con nombre de X y Y.</h4></font>

<font color='3A95B5'><h4>Donde:</h4></font>

<font color='3A95B5'><h4>Y = Norte(metros) o Latitud(grados)</h4></font>

<font color='3A95B5'><h4>X = Este(metros) o Longitud(grados)</h4></font>

<font color='3A95B5'><h4>Este código carga un archivo CSV con coordenadas, luego convierte estas coordenadas entre sistemas de referencia (UTM a Geográficas o viceversa). Las coordenadas convertidas se guardan en un nuevo archivo CSV que contiene solo las columnas originales y las columnas convertidas. El archivo resultante se descarga automáticamente al finalizar el proceso.</h4></font>

---


In [1]:
# Librerías para preparar entorno
import subprocess
import sys

# Función para instalar paquetes si no están instalados
def install_if_not_installed(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Lista de paquetes a verificar e instalar
packages = ["pandas", "xlsxwriter", "pyproj", "google.colab"]

print("Preparando entorno...")
# Revisar e instalar los paquetes necesarios si no están ya instalados
for package in packages:
    install_if_not_installed(package)
print("Entorno listo.")

# Librerías necesarias para procesamiento de datos
import pandas as pd
import xlsxwriter
from pyproj import CRS, Transformer
from google.colab import files

# @markdown **<font color='729D2D'><h3>Subir archivo CSV</h3></font>**

# Carga del archivo
uploaded = files.upload()

for filename in uploaded.keys():
    df = pd.read_csv(filename)
    print(f"Archivo {filename} subido y cargado:")
    print(df.head())

Preparando entorno...
Entorno listo.


Saving CoordenadasUTM_17S_WGS84.csv to CoordenadasUTM_17S_WGS84.csv
Archivo CoordenadasUTM_17S_WGS84.csv subido y cargado:
             X            Y
0  473745.9108  9529736.988
1  473745.9108  9529752.038
2  473746.9999  9529752.038
3  473746.9999  9529787.532
4  473746.9999  9529798.869


In [ ]:
# @title  { display-mode: "form" }
# @markdown **<font color='729D2D'><h3>SCR de entrada: Coordenadas Inicio</h3></font>**<br>
# Parámetros de entrada
ENTRADA_SISTEMA = "UTM"  # @param ["UTM", "GEOGRAFICAS"]
ENTRADA_ZONA = "17"  # @param {type:"string"}
ENTRADA_HEMISFERIO = "SUR"  # @param ["SUR", "NORTE"]
ENTRADA_DATUM = "WGS84"  # @param ["WGS84"]

# @markdown **<font color='729D2D'><h3>SCR Objetivo: Coordenadas transformadas</h3></font>**<br>
# Parámetros de salida
SALIDA_SISTEMA = "GEOGRAFICAS"  # @param ["UTM", "GEOGRAFICAS"]
SALIDA_ZONA = "17"  # @param {type:"string"}
SALIDA_HEMISFERIO = "SUR"  # @param ["SUR", "NORTE"]
SALIDA_DATUM = "WGS84"  # @param ["WGS84"]

# Definir nombres de columnas basados en el sistema de salida
if SALIDA_SISTEMA == "UTM":
    colX = "ESTE"
    colY = "NORTE"
else:
    colX = "LATITUD"
    colY = "LONGITUD"

# Definir el nombre del archivo de salida
if ENTRADA_SISTEMA == "GEOGRAFICAS" and SALIDA_SISTEMA == "GEOGRAFICAS":
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + ".xlsx"
elif ENTRADA_SISTEMA == "GEOGRAFICAS" and SALIDA_SISTEMA == "UTM":
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + "_" + SALIDA_ZONA + SALIDA_HEMISFERIO[0] + ".xlsx"
elif ENTRADA_SISTEMA == "UTM" and SALIDA_SISTEMA == "GEOGRAFICAS":
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_" + ENTRADA_ZONA + ENTRADA_HEMISFERIO[0] + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + ".xlsx"
else:
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_" + ENTRADA_ZONA + ENTRADA_HEMISFERIO[0] + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + "_" + SALIDA_ZONA + SALIDA_HEMISFERIO[0] + ".xlsx"

# Imprimir los valores de entrada y salida
print(f"Sistema de entrada: {ENTRADA_SISTEMA}")
print(f"Zona de entrada: {ENTRADA_ZONA} {ENTRADA_HEMISFERIO}")
print(f"Datum de entrada: {ENTRADA_DATUM}")
print(f"Sistema de salida: {SALIDA_SISTEMA}")
print(f"Zona de salida: {SALIDA_ZONA} {SALIDA_HEMISFERIO}")
print(f"Datum de salida: {SALIDA_DATUM}")
print(f"Nombre del archivo de salida: {name_file}")

# Función para definir el CRS
def definir_crs(sistema, zona=None, hemisferio=None):
    if sistema == "UTM":
        south = " +south" if hemisferio == "SUR" else ""
        crs = f"+proj=utm +zone={zona}{south} +datum=WGS84 +units=m +no_defs"
    else:
        crs = "EPSG:4326"
    return crs

# Función para convertir coordenadas
def convertir_coordenadas(df, entrada_sistema, entrada_zona, entrada_hemisferio, salida_sistema, salida_zona, salida_hemisferio):
    crs_in = definir_crs(entrada_sistema, entrada_zona, entrada_hemisferio)
    crs_out = definir_crs(salida_sistema, salida_zona, salida_hemisferio)

    # Crear un transformador
    transformer = Transformer.from_crs(CRS(crs_in), CRS(crs_out))

    # Convertir coordenadas
    if entrada_sistema == "UTM" and salida_sistema == "GEOGRAFICAS":
        df[colX], df[colY] = transformer.transform(df['X'].values, df['Y'].values)
    else:
        df[colX], df[colY] = transformer.transform(df['Y'].values, df['X'].values)

    return df

# Funciones para convertir a DMM y DMS
def grados_a_dmm(grados):
    signo = "-" if grados < 0 else ""
    grados_abs = abs(grados)
    grados_int = int(grados_abs)
    minutos_dec = (grados_abs - grados_int) * 60
    return f"{signo}{grados_int}° {minutos_dec:.6f}'"

def grados_a_dms(grados):
    signo = "-" if grados < 0 else ""
    grados_abs = abs(grados)
    grados_int = int(grados_abs)
    minutos = int((grados_abs - grados_int) * 60)
    segundos = (grados_abs - grados_int - minutos/60) * 3600
    return f"{signo}{grados_int}° {minutos}' {segundos:.6f}\""

# Crear un hipervínculo a Google Maps
def crear_link_google(lat, lon):
    return f"https://www.google.com/maps/search/?api=1&query={lat},{lon}"

# Uso de la función para convertir coordenadas
df_convertido = convertir_coordenadas(df, ENTRADA_SISTEMA, ENTRADA_ZONA, ENTRADA_HEMISFERIO, SALIDA_SISTEMA, SALIDA_ZONA, SALIDA_HEMISFERIO)
print("Coordenadas convertidas")

# Columnas adicionales si salida es GEOGRAFICAS
if SALIDA_SISTEMA == "GEOGRAFICAS":
    df_convertido['LATITUD DMM'] = df_convertido[colY].apply(grados_a_dmm)
    df_convertido['LONGITUD DMM'] = df_convertido[colX].apply(grados_a_dmm)
    df_convertido['LATITUD DMS'] = df_convertido[colY].apply(grados_a_dms)
    df_convertido['LONGITUD DMS'] = df_convertido[colX].apply(grados_a_dms)
    df_convertido['TO GOOGLE'] = df_convertido.apply(lambda row: crear_link_google(row['LATITUD'], row['LONGITUD']), axis=1)

# Columnas originales y las convertidas
df_salida = df[['X', 'Y', colX, colY]]

# Columnas adicionales si salida es GEOGRAFICAS
if SALIDA_SISTEMA == "GEOGRAFICAS":
    df_salida = df[['X', 'Y', colX, colY, 'LATITUD DMM', 'LONGITUD DMM', 'LATITUD DMS', 'LONGITUD DMS', 'TO GOOGLE']]

# Exportar a Excel con hipervínculos
with pd.ExcelWriter(name_file, engine='xlsxwriter') as writer:
    df_salida.to_excel(writer, index=False, sheet_name='Coordenadas')
    workbook = writer.book
    worksheet = writer.sheets['Coordenadas']
    if 'TO GOOGLE' in df_salida.columns:
        for row_num, url in enumerate(df_salida['TO GOOGLE'], start=1):
            worksheet.write_url(row_num, df_salida.columns.get_loc('TO GOOGLE'), url, string='To Google Maps')

print(f"Archivo guardado como {name_file}")

# Descarga
files.download(name_file)

Sistema de entrada: GEOGRAFICAS
Zona de entrada: 21 SUR
Datum de entrada: WGS84
Sistema de salida: UTM
Zona de salida: 17 SUR
Datum de salida: WGS84
Nombre del archivo de salida: GEOGRAFICAS_WGS84_A_UTM_WGS84_17S.csv
Coordenadas convertidas
Archivo guardado como GEOGRAFICAS_WGS84_A_UTM_WGS84_17S.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# <font color='729D2D'><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExbHduN3M0MWw1MThzYWVwaGw3cDFsZG92ODJ5Yzc4b3lnNjM4Y2ptMyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/qZb3YFfAf8YRgWqaVV/giphy.webp" width="7%"> <strong>Support My Work</strong></font>

<p align="left">
  <a href="https://www.buymeacoffee.com/henryzumaeta" target="_blank">
    <img src="https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/BuyMeCoffee_77px.png" alt="Buy Me A Coffee" style="height: 42px; width: 151.9px;">
  </a>
  <a href="https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/QR_yape.jpg" target="_blank">
    <img src="https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/Yape_77px.png" alt="Yape" style="height: 42px; width: 151.9px;">
  </a>
</p>

# <font color='729D2D'><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExaXQ5eHVhZGx4djZhcW42ZDR3Y3dnaWY5NjNwbGs3MGt0YXl1cWM0cSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9cw/1MyJr6oe9sDCCXs5ti/giphy.webp" width="8%"> <strong>Connect With Me</strong></font>

<p align="left">
  <a href="https://www.linkedin.com/in/henryzumaeta/" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=64154&format=png&color=000000" alt="linkedin" height="50" width="50" /></a>
  <a href="https://twitter.com/henryzumaeta" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=64156&format=png&color=000000" alt="twitter" height="50" width="50" /></a>
  <a href="https://www.instagram.com/henryzumaeta/" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=hFoVFpm6gl9A&format=png&color=000000" alt="instagram" height="50" width="50" /></a>
  <a href="https://api.whatsapp.com/send?phone=51963719768&text=%F0%9F%91%8B%F0%9F%91%A8%E2%80%8D%F0%9F%92%BB" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=108636&format=png&color=000000" alt="whatsapp" height="50" width="50" /></a>
  <a href="mailto:henry.zumaeta.l@uni.pe" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=6QtoKjRma1Cq&format=png&color=000000" alt="email" height="50" width="50" /></a>
</p>

# <font color='729D2D'><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExeHM0YnNpMnJrYWJrd243c2J5bW41d2Z4c2xoNzM2YjU0N3dsZmhvayZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/Xd1RbiqD2eL0YCHIjv/giphy.webp" width="6%"> <strong>Warning</strong></font>

<font color='FFD700'><h4>El autor no se hace responsable del mal uso del formulario y script.</h4></font>